In [ ]:
!git clone https://github.com/soroush-abbasi/289L_AdvDL.git

Cloning into '289L_AdvDL'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 14.33 KiB | 4.78 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
cd 289L_AdvDL

/content/289L_AdvDL


In [ ]:
!pip install timm==0.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import argparse
import datetime
import os
import sys
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn

from timm.models import create_model

from engine import train_one_epoch, train_one_epoch_distillation, evaluate
from utils import get_training_dataloader, get_test_dataloader
import models

# Q1

In [ ]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=False,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

assert (shots*num_classes == len(cifar10_training_loader.dataset))

cifar10_test_loader = get_test_dataloader(
    MEAN,
    STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


Creating model: vit_base_patch16_224


100%|██████████| 170498071/170498071 [00:12<00:00, 13322896.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
number of params: 86567656


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)

    train_stats = evaluate(cifar10_training_loader, model, criterion, device)
    test_stats = evaluate(cifar10_test_loader, model, criterion, device)
    print(f"Accuracy of the network on the {len(cifar10_training_loader)} train images: {train_stats['acc1']:.1f}%")
    print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:10:12  loss: 7.3936 (7.3936)  time: 0.9807  data: 0.1432  max mem: 2096
Epoch: [1]  [100/625]  eta: 0:00:57  loss: 2.0924 (2.5162)  time: 0.0999  data: 0.0034  max mem: 3067
Epoch: [1]  [200/625]  eta: 0:00:44  loss: 1.9737 (2.3078)  time: 0.1000  data: 0.0037  max mem: 3067
Epoch: [1]  [300/625]  eta: 0:00:33  loss: 2.0648 (2.2227)  time: 0.1004  data: 0.0036  max mem: 3067
Epoch: [1]  [400/625]  eta: 0:00:23  loss: 1.9214 (2.1639)  time: 0.1009  data: 0.0036  max mem: 3067
Epoch: [1]  [500/625]  eta: 0:00:12  loss: 2.0792 (2.1346)  time: 0.1015  data: 0.0036  max mem: 3067
Epoch: [1]  [600/625]  eta: 0:00:02  loss: 1.9556 (2.1012)  time: 0.1020  data: 0.0037  max mem: 3067
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 1.8628 (2.0936)  time: 0.1018  data: 0.0036  max mem: 3067
Epoch: [1] Total time: 0:01:03 (0.1022 s / it)
Averaged stats: loss: 1.8628 (2.0936)
Test:  [  0/625]  eta: 0:03:13  loss: 1.7258 (1.7258)  acc1: 37.5000 (3

In [ ]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

Test:  [ 0/40]  eta: 0:00:46  loss: 1.8480 (1.8480)  acc1: 31.6406 (31.6406)  acc5: 84.7656 (84.7656)  time: 1.1592  data: 0.6386  max mem: 3158
Test:  [20/40]  eta: 0:00:11  loss: 1.7542 (1.7542)  acc1: 34.3750 (34.3192)  acc5: 87.8906 (87.7604)  time: 0.5548  data: 0.0311  max mem: 3158
Test:  [39/40]  eta: 0:00:00  loss: 1.7649 (1.7464)  acc1: 33.9844 (34.2300)  acc5: 87.1094 (87.5000)  time: 0.5336  data: 0.0311  max mem: 3158
Test: Total time: 0:00:22 (0.5605 s / it)
* Acc@1 34.230 Acc@5 87.500 loss 1.746
Throughput: 445.9432419403781


# Q2

In [ ]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

assert (shots*num_classes == len(cifar10_training_loader.dataset))

cifar10_test_loader = get_test_dataloader(
    MEAN,
    STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


Creating model: vit_base_patch16_224


Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth
100%|██████████| 330M/330M [00:01<00:00, 228MB/s]


Files already downloaded and verified
Files already downloaded and verified
number of params: 85806346


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)

    train_stats = evaluate(cifar10_training_loader, model, criterion, device)
    test_stats = evaluate(cifar10_test_loader, model, criterion, device)
    print(f"Accuracy of the network on the {len(cifar10_training_loader)} train images: {train_stats['acc1']:.1f}%")
    print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:02:27  loss: 2.3259 (2.3259)  time: 0.2363  data: 0.1196  max mem: 3158
Epoch: [1]  [100/625]  eta: 0:00:54  loss: 2.1496 (2.2712)  time: 0.1022  data: 0.0034  max mem: 3158
Epoch: [1]  [200/625]  eta: 0:00:43  loss: 2.0386 (2.1677)  time: 0.1024  data: 0.0035  max mem: 3158
Epoch: [1]  [300/625]  eta: 0:00:33  loss: 1.8758 (2.1013)  time: 0.1027  data: 0.0037  max mem: 3158
Epoch: [1]  [400/625]  eta: 0:00:23  loss: 1.7478 (2.0338)  time: 0.1024  data: 0.0037  max mem: 3158
Epoch: [1]  [500/625]  eta: 0:00:12  loss: 1.5791 (1.9594)  time: 0.1022  data: 0.0038  max mem: 3158
Epoch: [1]  [600/625]  eta: 0:00:02  loss: 1.2156 (1.8595)  time: 0.1030  data: 0.0042  max mem: 3158
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 1.4263 (1.8421)  time: 0.1026  data: 0.0041  max mem: 3158
Epoch: [1] Total time: 0:01:04 (0.1027 s / it)
Averaged stats: loss: 1.4263 (1.8421)
Test:  [  0/625]  eta: 0:01:23  loss: 0.9358 (0.9358)  acc1: 75.0000 (7

In [ ]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

Test:  [ 0/40]  eta: 0:00:51  loss: 0.4256 (0.4256)  acc1: 86.7188 (86.7188)  acc5: 99.2188 (99.2188)  time: 1.2898  data: 0.7756  max mem: 3781
Test:  [20/40]  eta: 0:00:11  loss: 0.4715 (0.4779)  acc1: 84.3750 (84.6912)  acc5: 99.6094 (99.3676)  time: 0.5580  data: 0.0309  max mem: 3781
Test:  [39/40]  eta: 0:00:00  loss: 0.4904 (0.4843)  acc1: 84.3750 (84.5200)  acc5: 99.6094 (99.4100)  time: 0.5345  data: 0.0311  max mem: 3781
Test: Total time: 0:00:22 (0.5660 s / it)
* Acc@1 84.520 Acc@5 99.410 loss 0.484
Throughput: 441.62117512078635


# Q3

In [ ]:
MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
CHECKPOINT_PATH = './checkpoint'
MODEL_NAME = 'vit_tiny_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0
shots = 1000

print(f"Creating model: {MODEL_NAME}")
model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

cifar10_training_loader = get_training_dataloader(
    MEAN,
    STD,
    num_workers=2,
    batch_size=16,
    shuffle=True,
    shots=shots
)

assert (shots*num_classes == len(cifar10_training_loader.dataset))

cifar10_test_loader = get_test_dataloader(
    MEAN,
    STD,
    num_workers=4,
    batch_size=256,
    shuffle=False
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


Creating model: vit_tiny_patch16_224


Downloading: "https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth" to /root/.cache/torch/hub/checkpoints/deit_tiny_patch16_224-a1311bcf.pth
100%|██████████| 21.9M/21.9M [00:00<00:00, 170MB/s]


Files already downloaded and verified
Files already downloaded and verified
number of params: 5526346


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        model, criterion, cifar10_training_loader,
        optimizer, device, epoch)

    train_stats = evaluate(cifar10_training_loader, model, criterion, device)
    test_stats = evaluate(cifar10_test_loader, model, criterion, device)
    print(f"Accuracy of the network on the {len(cifar10_training_loader)} train images: {train_stats['acc1']:.1f}%")
    print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:02:10  loss: 3.1066 (3.1066)  time: 0.2085  data: 0.1145  max mem: 3781
Epoch: [1]  [100/625]  eta: 0:00:20  loss: 2.0346 (2.2067)  time: 0.0377  data: 0.0036  max mem: 3781
Epoch: [1]  [200/625]  eta: 0:00:16  loss: 1.9969 (2.1122)  time: 0.0372  data: 0.0033  max mem: 3781
Epoch: [1]  [300/625]  eta: 0:00:12  loss: 2.0834 (2.0860)  time: 0.0390  data: 0.0035  max mem: 3781
Epoch: [1]  [400/625]  eta: 0:00:08  loss: 1.7889 (2.0483)  time: 0.0375  data: 0.0034  max mem: 3781
Epoch: [1]  [500/625]  eta: 0:00:04  loss: 1.7989 (2.0081)  time: 0.0379  data: 0.0034  max mem: 3781
Epoch: [1]  [600/625]  eta: 0:00:00  loss: 1.7886 (1.9721)  time: 0.0386  data: 0.0034  max mem: 3781
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 1.5823 (1.9576)  time: 0.0383  data: 0.0035  max mem: 3781
Epoch: [1] Total time: 0:00:23 (0.0382 s / it)
Averaged stats: loss: 1.5823 (1.9576)
Test:  [  0/625]  eta: 0:01:19  loss: 1.5226 (1.5226)  acc1: 37.5000 (3

In [ ]:
# Calculate througput
start_time = time.time()
test_stats = evaluate(cifar10_test_loader, model, criterion, device)
end_time = time.time()
num_samples = len(cifar10_test_loader.dataset)
throughput = num_samples / (end_time - start_time)
print("Throughput: {}".format(throughput))

Test:  [ 0/40]  eta: 0:00:35  loss: 0.9639 (0.9639)  acc1: 66.0156 (66.0156)  acc5: 96.8750 (96.8750)  time: 0.8943  data: 0.7615  max mem: 3781
Test:  [20/40]  eta: 0:00:03  loss: 0.9031 (0.8991)  acc1: 67.1875 (67.6525)  acc5: 97.2656 (97.4702)  time: 0.1650  data: 0.0329  max mem: 3781
Test:  [39/40]  eta: 0:00:00  loss: 0.8795 (0.8887)  acc1: 66.7969 (67.7400)  acc5: 98.0469 (97.6300)  time: 0.1586  data: 0.0327  max mem: 3781
Test: Total time: 0:00:07 (0.1815 s / it)
* Acc@1 67.740 Acc@5 97.630 loss 0.889
Throughput: 1376.0974434723805


# Q4

In [ ]:
# Step 1: Train the teacher

MODEL_NAME = 'vit_base_patch16_224'
num_classes = 10
EPOCHS = 5
LR = 0.0001
WD = 0.0

print(f"Creating model: {MODEL_NAME}")
teacher = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
teacher = teacher.to(device)



criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(teacher.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in teacher.parameters() if p.requires_grad)
print('number of params:', n_parameters)


Creating model: vit_base_patch16_224
number of params: 85806346


In [ ]:
print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch(
        teacher, criterion, cifar10_training_loader,
        optimizer, device, epoch)

    if epoch % 10 == 9:
        test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")


Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:02:14  loss: 0.4484 (0.4484)  time: 0.2148  data: 0.1134  max mem: 3781
Epoch: [1]  [100/625]  eta: 0:00:54  loss: 0.2361 (0.2730)  time: 0.1028  data: 0.0036  max mem: 3781
Epoch: [1]  [200/625]  eta: 0:00:43  loss: 0.2619 (0.2809)  time: 0.1027  data: 0.0036  max mem: 3781
Epoch: [1]  [300/625]  eta: 0:00:33  loss: 0.1310 (0.2651)  time: 0.1021  data: 0.0036  max mem: 3781
Epoch: [1]  [400/625]  eta: 0:00:23  loss: 0.1814 (0.2737)  time: 0.1014  data: 0.0035  max mem: 3781
Epoch: [1]  [500/625]  eta: 0:00:12  loss: 0.2724 (0.2758)  time: 0.1010  data: 0.0035  max mem: 3781
Epoch: [1]  [600/625]  eta: 0:00:02  loss: 0.1551 (0.2791)  time: 0.1009  data: 0.0034  max mem: 3781
Epoch: [1]  [624/625]  eta: 0:00:00  loss: 0.1731 (0.2786)  time: 0.1011  data: 0.0036  max mem: 3781
Epoch: [1] Total time: 0:01:03 (0.1022 s / it)
Averaged stats: loss: 0.1731 (0.2786)
Epoch: [2]  [  0/625]  eta: 0:02:14  loss: 0.2355 (0.2355)  time: 0.214

In [ ]:
# save finetuned teacher model
torch.save(teacher.state_dict(), './teacher.pth')

In [ ]:
import utils
import torch.nn.functional as F
import math
def train_one_epoch_distillation(teacher, student, criterion,
                    data_loader, optimizer,
                    device: torch.device, epoch, alpha=1.0, temp=1.0):
    teacher.eval()
    student.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 100
    kl_loss = torch.nn.KLDivLoss(reduction="batchmean")

    for samples, targets in metric_logger.log_every(data_loader, print_freq, header):
        samples = samples.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)


        with torch.cuda.amp.autocast():
            outputs = student(samples)
            outputs_teacher = teacher(samples)

            # Imeplemet knowledge distillation loss here
            # loss = criterion(outputs, F.softmax(outputs_teacher, 0))
            loss = kl_loss(F.log_softmax(outputs, 0), F.softmax(outputs_teacher, 0))
            # print(loss)

            # *****************************************
        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        torch.cuda.synchronize()
        metric_logger.update(loss=loss_value)

    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}


In [ ]:

teacher = create_model(
        'vit_base_patch16_224',
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
teacher = teacher.to(device)
teacher.load_state_dict(torch.load('./teacher.pth'))

test_stats = evaluate(cifar10_test_loader, teacher, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

# Train the student
for p in teacher.parameters():
    p.requires_grad = False

MODEL_NAME = 'vit_tiny_patch16_224'

model = create_model(
        MODEL_NAME,
        pretrained=True,
        num_classes=10,
        img_size=224)
device = 'cuda:0' # device = 'cpu'
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)


n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('number of params:', n_parameters)


print(f"Start training for {EPOCHS} epochs")

for epoch in range(1, EPOCHS+1):
    train_stats = train_one_epoch_distillation(
        teacher, model, criterion, cifar10_training_loader,
        optimizer, device, epoch, alpha=2.0, temp=1.0)
    if epoch % 2 == 1:
        test_stats = evaluate(cifar10_test_loader, model, criterion, device)
        print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

test_stats = evaluate(cifar10_test_loader, model, criterion, device)
print(f"Accuracy of the network on the {len(cifar10_test_loader)} test images: {test_stats['acc1']:.1f}%")

Test:  [ 0/40]  eta: 0:00:50  loss: 0.6219 (0.6219)  acc1: 84.7656 (84.7656)  acc5: 98.8281 (98.8281)  time: 1.2650  data: 0.7541  max mem: 3781
Test:  [20/40]  eta: 0:00:11  loss: 0.6110 (0.6157)  acc1: 83.2031 (83.3147)  acc5: 98.8281 (98.9397)  time: 0.5487  data: 0.0319  max mem: 3781
Test:  [39/40]  eta: 0:00:00  loss: 0.6504 (0.6160)  acc1: 82.8125 (83.1000)  acc5: 98.8281 (99.0200)  time: 0.5264  data: 0.0312  max mem: 3781
Test: Total time: 0:00:22 (0.5571 s / it)
* Acc@1 83.100 Acc@5 99.020 loss 0.616
Accuracy of the network on the 40 test images: 83.1%
number of params: 5526346
Start training for 5 epochs
Epoch: [1]  [  0/625]  eta: 0:02:45  loss: 1.3984 (1.3984)  time: 0.2650  data: 0.1144  max mem: 3781
Epoch: [1]  [100/625]  eta: 0:00:31  loss: 1.0820 (1.2168)  time: 0.0578  data: 0.0034  max mem: 3781
Epoch: [1]  [200/625]  eta: 0:00:25  loss: 1.0549 (1.1470)  time: 0.0585  data: 0.0036  max mem: 3781
Epoch: [1]  [300/625]  eta: 0:00:19  loss: 1.0004 (1.1048)  time: 0.058